In [1]:
import os, sys
# for accessing src, stan, etc.
sys.path.append(os.path.abspath(os.path.join("../..")))

import bayesflow as bf
import numpy as np
from src.AmortizedMixture import *
from src.models.NormalHmm import *


import matplotlib.pyplot as plt

/Users/simonkucharsky/projects/bayesflow/amortized-mixture/.venv/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
model = NormalHmm(n_cls = 2, n_obs=[100, 100], separation=3.0)
model.prior_means = [0.0, 0.0, -1.5, 1.00]
model.prior_sds = [1.14,  1.14, 1.0, 0.65]

In [3]:
summary_net = bf.networks.SequenceNetwork(summary_dim=model.n_par*2)
inference_net = DependentClassificator(num_outputs=model.n_cls, bidirectional=False)
amortized_filtering = AmortizedMixture(
    inference_net=inference_net, 
    global_summary_net=summary_net)

In [4]:
trainer = bf.trainers.Trainer(amortizer=amortized_filtering, generative_model=model, checkpoint_path="checkpoints/filtering", configurator=lambda x: x)

INFO:root:Initialized empty loss history.
INFO:root:Initialized networks from scratch.
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [5]:
%time
h = trainer.train_online(epochs=50, iterations_per_epoch=1000, batch_size=32)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.1 µs


Training epoch 3:  55%|█████▌    | 550/1000 [01:21<01:07,  6.67it/s, Epoch: 3, Iter: 550,Loss: 10.970,Avg.Loss: 12.518,LR: 4.97E-04]

In [ ]:
f=bf.diagnostics.plot_losses(h)